# 🧭 AI Travel Planner Chatbot

An **AI travel planner chatbot** solves fragmented, time-consuming trip planning by delivering **instant, personalized itineraries** with a **safe, moderated conversational experience**.  
It reduces effort for travelers and cost for businesses while enabling continuous, iterative refinement of plans.

---

## 🧩 Problem Context

Travel planning is scattered across multiple tabs — flights, stays, activities, visas, insurance, and logistics — creating **cognitive load** and **decision fatigue**.  
Traditional service desks cannot provide **real-time, 24/7 guidance** at scale, and generic search results often miss personal constraints like **budget caps**, **dietary needs**, **accessibility**, or **group dynamics**.

---

## 💡 Why This Chatbot Is Needed

**For travelers:**
- Expect immediate answers and personalized recommendations.
- Want transparent trade-offs without endless searches.

**For businesses:**
- Need to handle surges in inquiries efficiently.
- Reduce support costs and convert interest into bookings faster.

> 💬 A conversational interface gathers requirements naturally, resolves ambiguities, and turns inputs into **actionable plans** with clear next steps.

---

## 🎯 Project Objectives

1. **Collect** essential trip requirements through guided questions and light validation.  
2. **Generate** day-by-day itineraries including routes, accommodations, activities, and budget guidance.  
3. **Maintain** a moderation layer to ensure safe, professional interactions.  
4. **Support** iterative feedback until user satisfaction, ending gracefully on explicit exit.

---

## 🧱 Scope of the Solution

- **Conversational requirement gathering:** destination, dates, group size, budget, accommodation style, interests, constraints.  
- **Itinerary generation:** daily schedules, transport, stay suggestions, tips, and contingency notes.  
- **Feedback loop:** apply requested changes and re-issue updated plans.  
- **Guardrails:** moderation on inputs/outputs before display.

---

## 🌟 Key Benefits

- **24/7 availability** → reduces wait times globally.  
- **Personalization** → aligns plans with user needs and constraints.  
- **Efficiency** → compresses planning time from hours to minutes.  
- **Consistency** → standard quality across destinations.  
- **Scalability** → supports many travelers simultaneously.  
- **Cost reduction** → automates repetitive support tasks.  
- **Conversion lift** → clear itineraries drive bookings.  
- **Continuous improvement** → learns from feedback.  
- **Accessibility** → supports multiple languages and simple guidance.  
- **Compliance & safety** → enforced through moderation.

---

## 🧳 Importance for Travelers

- **Faster decision-making** with transparent trade-offs (time vs. cost).  
- **Reduced anxiety** via structured steps and confirmations.  
- **Higher satisfaction** by matching goals, pace, and preferences.

---

## 🏢 Importance for Businesses

- **Lower acquisition and service costs** through automation.  
- **Improved retention** via proactive, personalized support.  
- **Better insights** from aggregated, consented data on preferences and needs.

---

## ⚙️ Core Capabilities in This Project

1. **Welcome and context setting** to build trust and explain process.  
2. **Requirement gathering** with sequential, validated questioning.  
3. **Plan generation** covering daily schedules, budget, and logistics.  
4. **Moderation layer** intercepting unsafe or noncompliant content.  
5. **Satisfaction check** with iterative modification until confirmed.

---

## 🧠 Differentiators

- **Conversation-first workflow** vs. form-based tools → improves engagement.  
- **Built-in guardrails** → ensure safety without loss of usability.  
- **Close-the-loop design** → verifies satisfaction, not just completion.

---

## 📊 Success Metrics (Examples)

- Requirement-collection **completion rate** and **average planning time**.  
- **Plan usefulness rating** and **iteration count to satisfaction**.  
- **Booking conversion** or **handoff success rate**.  
- **Containment rate** of routine queries (no human escalation).  
- **Moderation intervention rate** and **zero-incident count**.

---

## ⚠️ Risks and Mitigations

| **Risk** | **Mitigation Strategy** |
|-----------|--------------------------|
| Hallucinated or outdated suggestions | Scoped prompts, disclaimers, retrieval from reliable sources |
| Over-automation | Provide human handoff for complex cases |
| Privacy concerns | Minimal data collection, transparent use policy |
| API rate limits or quota issues | Backoff mechanisms, token budgeting, usage monitoring |

---

> ✨ **In summary:**  
> The AI Travel Planner Chatbot combines conversational intelligence, personalization, and safety to make travel planning **smarter, faster, and more human-friendly** — for both travelers and businesses.


# Workflow of the AI Travel Planner :<br>
1. **Welcome message:** Initialize the session, explain capabilities, and prompt the traveler to begin, establishing context for subsequent questions
2. **Gather requirements:** Collect destination, dates, party size, budget, accommodation preferences, interests, constraints, and transport preferences via sequential prompts.
3. **Moderation check:** Validate incoming inputs against a safety layer to filter harmful or noncompliant content before using them downstream.
4. **Plan via ChatGPT API:** Transform validated requirements into a structured itinerary including daily plan, stay and transport suggestions, budget guidance, and local tips.
5. **Present plan:** Return the generated itinerary to the traveler in clear sections and confirm understanding of major trade‑offs.
6. **Satisfaction decision:** Ask whether the traveler is satisfied, branching on the response to either modification or completion.
7. **If “No”:** Loop back to the planning stage with requested changes (e.g., tighter budget, different neighborhoods, activity swaps) and regenerate the plan.
8. **If “Yes”:** Proceed to end state, optionally offering next‑step actions such as exporting, sharing, or booking hand‑off.
9. **End:** Close the session with a final confirmation, persisting the last accepted plan and stopping further processing.

![AI Travel Planner Architecture](AI_Travel_Chatbot_Architecture.png)

# Implementation :
![AI Travel Planner Architecture](generated-image.png)

### Install the required package

In [7]:

!pip install openai

### Import required modules

In [8]:
import openai
import os

### Define the chatbot class

In [9]:
class AITravelPlanner:
    def __init__(self, api_key: str):
        self.client = openai.OpenAI(api_key=api_key)
        self.conversation_history = []
        self.user_info = {}
        self.travel_plan = ""
        self.moderation_enabled = True

        # Travel planning questions
        self.travel_questions = [
            "Where would you like to travel to? (destination)",
            "What are your travel dates? (departure and return dates)",
            "How many people will be traveling?",
            "What's your approximate budget for this trip?",
            "What type of accommodation do you prefer? (hotel, hostel, Airbnb, etc.)",
            "What activities or experiences interest you most?",
            "Do you have any dietary restrictions or special requirements?",
            "What's your preferred mode of transportation?"
        ]
        self.current_question_index = 0
        self.gathering_info = False

    def moderate_content(self, text: str) -> bool:
        if not self.moderation_enabled:
            return True
        try:
            response = self.client.moderations.create(input=text)
            return not response.results[0].flagged
        except Exception as e:
            print(f"Moderation error: {e}")
            return True  # Default to safe if moderation fails

    def get_chatgpt_response(self, user_message: str, system_prompt: str = None) -> str:
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.extend(self.conversation_history)
        messages.append({"role": "user", "content": user_message})
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                max_tokens=500,
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Sorry, I encountered an error: {e}"

    def welcome_message(self) -> str:
        return """
🌟 Welcome to AI Travel Planner! 🌟

Hello! I'm your personal AI travel assistant, and I'm excited to help you plan your perfect trip! 
✈️ Whether you're looking for a relaxing beach vacation, an adventurous mountain trek, or a cultural city exploration, I'm here to create a customized travel plan just for you.

I'll ask you a few questions to understand your preferences and requirements, then create a detailed travel itinerary tailored to your needs.

Let's start planning your amazing journey! Are you ready to begin?
        """.strip()

    def ask_travel_questions(self) -> str:
        if self.current_question_index < len(self.travel_questions):
            return f"Question {self.current_question_index + 1}: {self.travel_questions[self.current_question_index]}"
        else:
            return "Great! I have all the information I need. Let me create your travel plan..."

    def process_travel_answer(self, answer: str) -> None:
        self.user_info[f"question_{self.current_question_index + 1}"] = {
            "question": self.travel_questions[self.current_question_index],
            "answer": answer
        }
        self.current_question_index += 1

    def generate_travel_plan(self) -> str:
        user_requirements = "\n".join([
            f"{info['question']}: {info['answer']}" 
            for info in self.user_info.values()
        ])
        system_prompt = """
You are an expert travel planner. Based on the user's requirements, create a detailed,
personalized travel itinerary. Include:
- Daily schedule with activities
- Accommodation suggestions
- Transportation options
- Budget breakdown
- Local tips and recommendations
- Emergency contacts and important information

Make it comprehensive but easy to follow.
        """
        planning_request = f"""
Please create a detailed travel plan based on these requirements:
{user_requirements}

Make it personalized and include practical details.
        """
        plan = self.get_chatgpt_response(planning_request, system_prompt)
        self.travel_plan = plan
        return plan

    def ask_for_satisfaction(self) -> str:
        return """
Are you satisfied with this travel plan? 
- Type 'yes' if you're happy with it
- Type 'no' and tell me what you'd like to change
- Type 'exit' to end our conversation
        """.strip()

    def modify_travel_plan(self, changes: str) -> str:
        system_prompt = f"""
You are a travel planner. The user wants to modify their travel plan. 
Here's the current plan: {self.travel_plan}

Make the requested changes and provide an updated plan.
        """
        modification_request = f"Please modify the travel plan with these changes: {changes}"
        modified_plan = self.get_chatgpt_response(modification_request, system_prompt)
        self.travel_plan = modified_plan
        return modified_plan

    def run_chatbot(self):
        print("="*60)
        print("AI TRAVEL PLANNER CHATBOT")
        print("="*60)

        # Step 1: Welcome message
        print(self.welcome_message())

        # Wait for user to start
        user_input = input("\nUser: ").strip()
        if not self.moderate_content(user_input):
            print("Bot: I'm sorry, but I can't process that message. Please keep our conversation appropriate.")
            return

        # Step 2: Gather travel information
        self.gathering_info = True
        print("\nBot: Excellent! Let's gather some information to plan your perfect trip.\n")

        while self.current_question_index < len(self.travel_questions):
            question = self.ask_travel_questions()
            print(f"Bot: {question}")

            user_answer = input("User: ").strip()
            if not self.moderate_content(user_answer):
                print("Bot: I'm sorry, but I can't process that response. Please provide a different answer.")
                continue
            if user_answer.lower() == 'exit':
                print("Bot: Thank you for using AI Travel Planner. Have a great day!")
                return

            self.process_travel_answer(user_answer)
            self.conversation_history.append({"role": "user", "content": user_answer})

        # Step 3: Generate travel plan
        print("\nBot: Perfect! I have all the information I need. Let me create your personalized travel plan...\n")
        print("🔄 Generating your travel plan... Please wait...")
        travel_plan = self.generate_travel_plan()
        print(f"\nBot: Here's your personalized travel plan:\n\n{travel_plan}")

        # Step 4 & 5: Satisfaction check and modifications
        satisfied = False
        while not satisfied:
            print(f"\nBot: {self.ask_for_satisfaction()}")
            user_response = input("User: ").strip().lower()
            if not self.moderate_content(user_response):
                print("Bot: I'm sorry, but I can't process that message. Please respond appropriately.")
                continue
            if user_response == 'exit':
                break
            elif user_response == 'yes':
                satisfied = True
                print("Bot: Wonderful! I'm glad you're happy with your travel plan. Have an amazing trip! 🎉")
            elif user_response == 'no':
                print("Bot: No problem! What would you like me to change in your travel plan?")
                changes = input("User: ").strip()
                if not self.moderate_content(changes):
                    print("Bot: I'm sorry, but I can't process that request. Please provide different feedback.")
                    continue
                print("Bot: Let me update your travel plan with those changes...")
                modified_plan = self.modify_travel_plan(changes)
                print(f"\nBot: Here's your updated travel plan:\n\n{modified_plan}")
            else:
                print("Bot: Please respond with 'yes', 'no', or 'exit'.")
        print("\nBot: Thank you for using AI Travel Planner! Safe travels! ✈️🌎")
        print("="*60)


### Initialize and run the chatbot

In [10]:

api_key = 'Your api key'

chatbot = AITravelPlanner(api_key)
chatbot.run_chatbot()


AI TRAVEL PLANNER CHATBOT
🌟 Welcome to AI Travel Planner! 🌟

Hello! I'm your personal AI travel assistant, and I'm excited to help you plan your perfect trip! 
✈️ Whether you're looking for a relaxing beach vacation, an adventurous mountain trek, or a cultural city exploration, I'm here to create a customized travel plan just for you.

I'll ask you a few questions to understand your preferences and requirements, then create a detailed travel itinerary tailored to your needs.

Let's start planning your amazing journey! Are you ready to begin?



User:  yes



Bot: Excellent! Let's gather some information to plan your perfect trip.

Bot: Question 1: Where would you like to travel to? (destination)


User:  Rajasthan


Bot: Question 2: What are your travel dates? (departure and return dates)


...

Bot: No problem! What would you like me to change in your travel plan?


User:  i want to taste the food, local food, sweets , can you had famous restraunts in the planner


Bot: Let me update your travel plan with those changes...

Bot: Here's your updated travel plan:

### Updated Travel Itinerary for Rajasthan Trip

**Destination:** Rajasthan, India  
**Travel Dates:** First week of November  
**Number of Travelers:** 6 people  
**Approximate Budget:** 20000 per person  

#### Day 1: Arrival in Jaipur
- **Morning:** Arrive in Jaipur by private vehicle.
- **Afternoon:** Check into a budget-friendly hotel or hostel.
- **Evening:** Explore the local markets and enjoy dinner at a non-veg restaurant to taste local Rajasthani cuisine.

**Accommodation:** Budget hotel or hostel in Jaipur  
**Budget Breakdown:** Accommodation - 2000 per person  

#### Day 2: Jaipur Sightseeing
- **Morning:** Visit Amber Fort and enjoy an elephant ride.
- **Afternoon:** Explore City Palace, Jantar Mantar, and Hawa Mahal.
- **Evening:** Shop for souvenirs at local markets.

**Accommodation:** Same as Day 1  
**Budget Breakdown:** Sightseeing - 1500 per person  

#### Day 3: Jaipu

User:  yes


Bot: Wonderful! I'm glad you're happy with your travel plan. Have an amazing trip! 🎉

Bot: Thank you for using AI Travel Planner! Safe travels! ✈️🌎


## ✈️ **Future Extensions**

- 🧭 **Live Inventory & Pricing Integration:**  
  Seamless connection for **flights, stays, and activities** ensuring real-time availability and accurate pricing.

- ⚡ **Real-Time Alerts:**  
  Get instant notifications for **delays, weather updates, or policy changes** with **automatic re-planning**.

- 👥 **Multi-Traveler Coordination:**  
  Enables **shared itineraries**, synchronized updates, and group travel planning.

- 🗣️ **Voice & Multilingual Interface:**  
  Introduces **voice commands**, **multilingual support**, and improved **accessibility** for diverse users.

---

## 🧾 **Summary**

> This AI travel chatbot transforms trip planning into a **guided, conversational experience**.  
> It **captures user preferences**, **suggests tailored itineraries**, and **ensures safety** through moderated interactions.  
> The outcome is:
> - **Faster planning**
> - **Higher traveler confidence**
> - **Scalable, cost-efficient operations** for organizations offering travel services.
